# Dataset Inspection

In [ ]:
from __future__ import print_function, division

from pathlib2 import Path

import numpy as np
import pandas
import rospy

from vgn import from_voxel_coordinates
from vgn.dataset import Dataset
from vgn.grasp import Grasp
from vgn.utils.transform import Rotation, Transform
from vgn_ros import vis


finger_depth = 0.05
size = 6.0 * finger_depth
voxel_size = size / 40.0

In [ ]:
dataset_dir = Path("../data/datasets/train")
csv_path = dataset_dir / "grasps.csv"

## Compute Statistics

In [ ]:
df = pandas.read_csv(csv_path)

positives = df[df["label"] == 1]
negatives = df[df["label"] == 0]

print("Number of samples:", len(df.index))
print("Number of positives:", len(positives.index))
print("Number of negatives:", len(negatives.index))

## Visualize Samples

Visualize random grasp samples in rviz.

In [ ]:
rospy.init_node("dataset_inspection")

In [ ]:
dataset = Dataset(dataset_dir, augment=False)

i = np.random.choice(len(dataset))
tsdf, (label, rotations, width), index = dataset[i]
grasp = Grasp(Transform(Rotation.from_quat(rotations[0]), index), width)

grasp = from_voxel_coordinates(grasp, voxel_size)

vis.clear()
vis.workspace(size)
vis.tsdf(tsdf.squeeze(), voxel_size)
vis.grasps([grasp], [float(label)], finger_depth)

## Balance Dataset

Discard a random subset of negative samples to ensure the same number of positive and negative grasp samples.

In [ ]:
df = pandas.read_csv(csv_path)
positives = df[df["label"] == 1]
negatives = df[df["label"] == 0]

i = np.random.choice(negatives.index, len(negatives.index) - len(positives.index), replace=False)
df = df.drop(i)

df.to_csv(csv_path, index=False)

In [ ]:
# DANGER ZONE - remove unreferenced TSDFs.
SIMULATE = True
names = df["tsdf"].values
for f in dataset_dir.iterdir():
    if f.suffix == ".npz" and f.name not in names:
        print("Removed ", f)
        if not SIMULATE:
            f.unlink()